In [0]:
%sql
Create catalog if not exists telecom_catalog_assign;

create schema if not exists telecom_catalog_assign.landing_zone;

create volume if not exists telecom_catalog_assign.landing_zone.landing_vol

In [0]:
dbutils.fs.mkdirs('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/')

dbutils.fs.mkdirs('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/')

dbutils.fs.mkdirs('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/')                  

In [0]:
dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv', '101,Arun,31,Chennai,PREPAID\n102,Meera,45,Bangalore,POSTPAID\n103,Irfan,29,Hyderabad,PREPAID\n104,Raj,52,Mumbai,POSTPAID\n105,27,Delhi,PREPAID\n106,Sneha,abc,Pune,PREPAID ',overwrite=True)

